In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy as np
import librosa
import glob
import pickle
from tqdm import tqdm

from MusicVectorizer import MusicVectorizer

import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import ruptures as rpt 

SAMPLE_RATE = 16000

In [ ]:
data_folders = ["90s_hiphop", "90s_rock", "2010s_pop", "classical", "country"]

In [ ]:
# load trajectories
trajectories = dict()
for folder in data_folders:
    trajectories[folder] = np.load(f"data/{folder}_trajectories.npy", allow_pickle=True).item()

In [ ]:
lengths = list()
for folder in data_folders:
    for file in trajectories[folder]:
        lengths.append(trajectories[folder][file].shape[0])

In [45]:
song = trajectories[data_folders[2]][list(trajectories[data_folders[2]].keys())[7]]

In [49]:
avg_song = np.empty((song.shape[0], song.shape[2]))

    # run the first layer of the model on song[i,:].flatten()
output = model.fc1(torch.tensor(song).flatten(start_dim=1).float()).detach().numpy()
avg_song = output.reshape(song.shape[0], song.shape[2])

    

In [ ]:
# total number of features that we have:
N = np.sum(lengths)
N

In [ ]:
def fig_ax(figsize=(15, 5), dpi=150):
    """Return a (matplotlib) figure and ax objects with given size."""
    return plt.subplots(figsize=figsize, dpi=dpi)

In [ ]:
song = trajectories[data_folders[2]][list(trajectories[data_folders[2]].keys())[7]]
song = song[:,12,:]
song.shape

In [ ]:
# Choose detection method
algo = rpt.KernelCPD(kernel="linear").fit(avg_song)
#algo = rpt.Dynp(model="l2").fit(song)

# Choose the number of changes (elbow heuristic)
n_bkps_max = 20  # K_max
# Start by computing the segmentation with most changes.
# After start, all segmentations with 1, 2,..., K_max-1 changes are also available for free.
_ = algo.predict(n_bkps_max)

array_of_n_bkps = np.arange(1, n_bkps_max + 1)


def get_sum_of_cost(algo, n_bkps) -> float:
    """Return the sum of costs for the change points `bkps`"""
    bkps = algo.predict(n_bkps=n_bkps)
    return algo.cost.sum_of_costs(bkps)


fig, ax = fig_ax((7, 4))
ax.plot(
    array_of_n_bkps,
    [get_sum_of_cost(algo=algo, n_bkps=n_bkps) for n_bkps in array_of_n_bkps],
    "-*",
    alpha=0.5,
)
ax.set_xticks(array_of_n_bkps)
ax.set_xlabel("Number of change points")
ax.set_title("Sum of costs")
ax.grid(axis="x")
ax.set_xlim(0, n_bkps_max + 1)

# Visually we choose n_bkps=5 (highlighted in red on the elbow plot)
n_bkps = 8
_ = ax.scatter([8], [get_sum_of_cost(algo=algo, n_bkps=8)], color="r", s=100)


In [ ]:
# Segmentation
bkps = algo.predict(n_bkps=n_bkps)
print(f"Change points: {bkps} (frame counts)")


In [43]:
# Crazy Torch Model
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First fully connected layer
      self.fc1 = nn.Linear(13 * 768, 768)
      
      # Get outputs
      self.fc2 = nn.Linear(768,5)
      
    def forward(self, x):
      outputs = F.relu(self.fc1(x))
      outputs = F.sigmoid(self.fc2(outputs))
      return outputs
    
model = Net()
model.load_state_dict(torch.load('models/model_50000.pt'))

<All keys matched successfully>